In [4]:
import numpy as np
import pandas as pd

import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score

from functools import reduce

from scipy.stats import ks_2samp
from scikitplot.metrics import plot_ks_statistic,plot_roc_curve

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',500)
pd.set_option('display.float_format',lambda x:'%.4f'%x)

## Cargar datos y categorizar variables

In [10]:
ruta='/home/oszwaldo/Documentos/Projects/Selectos/Unidad 2/Hojas de cálculo'
r=os.listdir('/home/oszwaldo/Documentos/Projects/Selectos/Unidad 2/Hojas de cálculo')[2]
arch= os.path.join(ruta,r)

In [11]:
df = pd.read_excel(arch)
print(df.shape)
df = df.loc[df['Edad 🎂']<=40].reset_index(drop=True)
df = df.loc[df['¿Cuántos cuartos para dormir tiene tu vivienda?']>0].reset_index(drop=True)
df = df.loc[df['¿A cuánto aproximadamente ascienden los ingresos de tu hogar(considerando a todos los que aportan)?']<=100000].reset_index(drop=True)
df = df.loc[df['¿En cuántos años lo terminaste?']<=5].reset_index(drop=True)
print(df.shape)

(228, 81)
(222, 81)


In [12]:
varc = ['Edad 🎂',
        '¿Cuánto tiempo dura (en minutos) el trayecto de tu casa a la facultad? ⏱',
        '¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti',
        '¿Cuántos son hombres?',
        '¿Cuántas son mujeres?',
        '¿Cuánto tiempo llevas viviendo ahí? (años)',
        '¿Cuántos cuartos para dormir tiene tu vivienda?',
        '¿Cuántos automóviles en total tienen las personas que conforman tu hogar?',
        '¿Cuántas horas trabajas a la semana?',
        '¿Qué promedio obtuviste en el bachillerato?',
        '¿En cuántos años lo terminaste?',
        '¿A cuánto aproximadamente ascienden los ingresos de tu hogar(considerando a todos los que aportan)?',
        '¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Deportes ⚽️🏀🏈]',
        '¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Actividades culturales 🎭🎬🎻]',
        '¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Actividades sociales (fiestas, reuniones con amigos, familia, etc.)]',
        '¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Entretenimiento personal (videojuegos, redes sociales, netflix, youtube, etc)]'
       ]

vard = ['Estado civil','¿Tienes hijos?',
        '¿Tienes alguna dificultad para? [👓 Ver]',
        '¿Tienes alguna dificultad para? [👂 Escuchar]',
        '¿Tienes alguna dificultad para? [💪 Realizar alguna actividad física]',
        '¿Dónde vives actualmente?',
        '¿En qué delegación o municipio?',
        '¿Qué medio de transporte utilizas para llegar a la facultad?',
        '¿Cuál de las siguientes opciones describe tu tipo de vivienda?',
        '¿Con quién vives? (Puedes seleccionar más de una)',
        '¿Cuál es el máximo nivel de estudios de tu papá?',
        '¿Cuál es el máximo nivel de estudios de tu mamá?',
        'En caso de que tengas smartphone ¿Qué sistema operativo tiene?',
        '¿Dónde vivirás mientras cursas tus estudios universitarios?',
        '¿De qué forma financiarás tus estudios universitarios?',
        '¿En dónde cursaste el bachillerato?','¿Cómo se llama la institución donde cursaste el bachillerato?',
        '¿Recibiste alguna beca?','¿Cuál fue el proceso de admisión por el que entraste a esta carrera?',
        'Escoger ésta carrera fue...','Al escoger la FES Acatlán ésta fue:',
        '¿Cuándo tomaste la decisión de lo que querías estudiar?'   
       ]
tgt = 'Carrera'

## Normalizar variables discretas

In [13]:
for v in vard:
    df[v].fillna('MISSING',inplace=True)

In [14]:
def normalizar(df,v,umbral=0.05):
    aux = df[v].value_counts(True).to_frame()
    aux['norm'] = np.where(aux[v]<umbral,'Otros',aux.index)
    grupo = aux.loc[aux['norm']=='Otros'][v].sum()
    if grupo<umbral:
        aux['norm'].replace({'Otros':aux.index[0]},inplace=True)
    aux.drop(v,axis=1,inplace=True)
    return v,aux.to_dict()['norm']

In [15]:
l_norm = list(map(lambda v:normalizar(df,v),vard)    )

In [16]:
for v,d in l_norm:
    df[f'n_{v}'] = df[v].replace(d)

In [17]:
varn = df.filter(like='n_').columns.tolist()

In [18]:
for v in varn:
    print(v,'\n')
    print(df[v].value_counts(True).sort_index())
    print('\n'*2)

n_Estado civil 

👤 Soltero           0.7793
👥 En una relación   0.2207
Name: n_Estado civil, dtype: float64



n_¿Tienes hijos? 

No   1.0000
Name: n_¿Tienes hijos?, dtype: float64



n_¿Tienes alguna dificultad para? [👓 Ver] 

No   0.5135
Si   0.4865
Name: n_¿Tienes alguna dificultad para? [👓 Ver], dtype: float64



n_¿Tienes alguna dificultad para? [👂 Escuchar] 

No   1.0000
Name: n_¿Tienes alguna dificultad para? [👂 Escuchar], dtype: float64



n_¿Tienes alguna dificultad para? [💪 Realizar alguna actividad física] 

No   1.0000
Name: n_¿Tienes alguna dificultad para? [💪 Realizar alguna actividad física], dtype: float64



n_¿Dónde vives actualmente? 

CDMX               0.3829
Estado de México   0.6171
Name: n_¿Dónde vives actualmente?, dtype: float64



n_¿En qué delegación o municipio? 

Naucalpan   0.0856
Otros       0.9144
Name: n_¿En qué delegación o municipio?, dtype: float64



n_¿Qué medio de transporte utilizas para llegar a la facultad? 

Transporte público (metro, metrobu

In [19]:
unarias = [v for v in varn if df[v].value_counts().shape[0]==1]

In [20]:
varn = [v for v in varn if v not in unarias]

## Discretización de variables continuas

In [21]:
l = []
for v in varc:
    for st in ['uniform','quantile']:
        for k in range(2,6):
            kb = KBinsDiscretizer(n_bins=k,
                                  strategy=st,
                                  encode='ordinal')
            
            miss, nomiss = df.loc[df[v].isnull()][[v]],df.loc[~df[v].isnull()][[v]]
            kb.fit(nomiss[[v]])
            nombre = f'd_{v}_{k}_{st[:3]}' 
            miss[nombre] = miss[v].fillna('MISSING')
            nomiss[nombre] = pd.cut(nomiss[v],bins=kb.bin_edges_[0],include_lowest=True).astype(str)
            l.append(pd.concat([nomiss[[nombre]],miss[[nombre]]]))

In [22]:
dfd = reduce(lambda x,y:pd.merge(x,y,left_index=True,right_index=True,how='outer'),l)

In [23]:
dfd.shape,df.shape

((222, 128), (222, 103))

## Juntar discretas, continuas y target

In [24]:
df = df[[tgt]+varn].merge(dfd,left_index=True,right_index=True,how='outer')

In [25]:
df[tgt] = (df[tgt] =='Actuaria').astype(int)

In [26]:
df[tgt].value_counts(1)

1   0.6802
0   0.3198
Name: Carrera, dtype: float64

## Calcular IV

In [27]:
v = 'n_¿Cuál de las siguientes opciones describe tu tipo de vivienda?'

In [28]:
def iv(df,v,tgt):
    aux = df[[v,tgt]].assign(n=1)
    aux = aux.pivot_table(index=v,
                    columns=tgt,
                    values='n',
                    fill_value=0,
                    aggfunc='sum')
    for i in range(2):
        aux[i]/=aux[i].sum()
    aux['woe'] = np.log(aux[0]/aux[1])
    aux['iv'] = (aux[0]-aux[1])*aux['woe']
    return v,aux['iv'].sum()
    

In [29]:
print("""<0.1 Débil
0.1-0.3 Media
0.3-0.5 Fuerte
>0.5 Muy fuerte
""")

<0.1 Débil
0.1-0.3 Media
0.3-0.5 Fuerte
>0.5 Muy fuerte



In [30]:
var = [v for v in df.columns if v != tgt]

In [49]:
ivdf = pd.DataFrame(map(lambda v:iv(df,v,tgt),var),columns=['variable','iv'])
ivdf = ivdf.sort_values(by='iv',ascending=False).reset_index(drop=True)
ivdf['iv'].replace({np.inf:np.nan,-np.inf:np.nan},inplace=True)
ivdf = ivdf.dropna().reset_index(drop=True)
ivdf['raiz'] = ivdf['variable'].map(lambda x:x.split('_')[1] if x[:2]=='d_' else x)
ivdf = ivdf.sort_values(by=['raiz','iv'],ascending=[1,0]).reset_index(drop=True)
ivdf['best'] = ivdf.groupby('raiz').cumcount()+1
ivdf = ivdf.loc[ivdf['best']==1].drop('best',axis=1).sort_values(by='iv',ascending=False).reset_index(drop=True)

In [50]:
best = ivdf.loc[ivdf['iv']>=0.1]['variable'].tolist()

In [51]:
best

['d_¿Qué promedio obtuviste en el bachillerato?_4_qua',
 'n_Escoger ésta carrera fue...',
 'n_¿Cuál fue el proceso de admisión por el que entraste a esta carrera?',
 'n_¿Cuándo tomaste la decisión de lo que querías estudiar?',
 'd_¿En cuántos años lo terminaste?_3_uni',
 'n_¿En dónde cursaste el bachillerato?',
 'd_¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Entretenimiento personal (videojuegos, redes sociales, netflix, youtube, etc)]_5_qua',
 'n_¿Recibiste alguna beca?',
 'n_Al escoger la FES Acatlán ésta fue:',
 'n_¿Cómo se llama la institución donde cursaste el bachillerato?',
 'd_¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti_5_qua',
 'd_¿Cuánto tiempo llevas viviendo ahí? (años)_5_qua',
 'n_¿Cuál es el máximo nivel de estudios de tu mamá?',
 'n_¿Cuál es el máximo nivel de estudios de tu papá?']

## Transformación WoE

In [52]:
X = df[best].copy()
y = df[[tgt]].copy()

In [53]:
Xt,Xv,yt,yv = train_test_split(X,y,train_size=0.7)

In [54]:
Xt[tgt] = yt

In [37]:
def woe(df,v,tgt):
    aux = df[[v,tgt]].assign(n=1)
    aux = aux.pivot_table(index=v,
                    columns=tgt,
                    values='n',
                    fill_value=0,
                    aggfunc='sum')
    for i in range(2):
        aux[i]/=aux[i].sum()
    aux['woe'] = np.log(aux[0]/aux[1])
    aux.drop(range(2),axis=1,inplace=True)
    return v,aux.to_dict()['woe']
    

In [55]:
mapa_woe = list(map(lambda v:woe(Xt,v,tgt),best))

In [56]:
for v,d in mapa_woe:
    Xt[f'w_{v}'] = Xt[v].replace(d)
    Xv[f'w_{v}'] = Xv[v].replace(d)

In [57]:
varw = [v for v in Xt.columns if v[:2]=='w_']

In [58]:
Xt = Xt[varw]
Xv = Xv[varw]

In [59]:
aux = pd.concat([Xt,Xv])
aux = aux.describe().T[['min']]
aux['min'] = np.abs(aux['min'])

In [61]:
quitar = aux.loc[np.isinf(aux['min'])].index.tolist()
quitar

[]

In [62]:
varw = [v for v in varw if v not in quitar]

In [63]:
Xt = Xt[varw]
Xv = Xv[varw]

## Entrenamiento del modelo

In [65]:
modelo = LogisticRegression()

In [64]:
Xt.describe()

,w_d_¿Qué promedio obtuviste en el bachillerato?_4_qua,w_n_Escoger ésta carrera fue...,w_n_¿Cuál fue el proceso de admisión por el que entraste a esta carrera?,w_n_¿Cuándo tomaste la decisión de lo que querías estudiar?,w_d_¿En cuántos años lo terminaste?_3_uni,w_n_¿En dónde cursaste el bachillerato?,"w_d_¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Entretenimiento personal (videojuegos, redes sociales, netflix, youtube, etc)]_5_qua",w_n_¿Recibiste alguna beca?,w_n_Al escoger la FES Acatlán ésta fue:,w_n_¿Cómo se llama la institución donde cursaste el bachillerato?,w_d_¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti_5_qua,w_d_¿Cuánto tiempo llevas viviendo ahí? (años)_5_qua,w_n_¿Cuál es el máximo nivel de estudios de tu mamá?,w_n_¿Cuál es el máximo nivel de estudios de tu papá?
count,155.0000,155.0000,155.0000,155.0000,155.0000,155.0000,155.0000,155.0000,155.0000,155.0000,155.0000,155.0000,155.0000,155.0000
mean,-0.1911,inf,-0.0442,-0.0407,0.0066,-0.0580,-0.0477,-0.0401,-0.0429,-0.0614,-0.0316,-0.0300,-0.0170,-0.0121
std,1.5276,nan,0.8603,0.5956,0.6210,0.6845,0.5528,0.5594,0.4940,0.6169,0.5048,0.4625,0.3438,0.2845
min,-1.6191,-0.3403,-0.4469,-1.2624,-0.1474,-0.5118,-0.7028,-0.3800,-0.7028,-1.8014,-0.7028,-0.5533,-0.4151,-0.2328
25%,-1.0082,-0.3403,-0.4469,-0.1020,-0.1474,-0.5118,-0.5061,-0.3800,-0.5895,0.0018,-0.4151,-0.5533,-0.2328,-0.2328
50%,-1.0082,-0.3403,-0.4469,-0.1020,-0.1474,-0.5118,-0.5061,-0.3800,0.3581,0.0018,-0.1050,-0.0897,-0.2328,-0.1856
75%,2.1876,-0.3403,-0.4469,0.3040,-0.1474,0.9199,0.5164,0.8745,0.3581,0.0018,0.0704,0.4152,-0.0097,0.4384
max,2.1876,inf,1.7821,1.7821,2.6294,1.2431,0.7635,0.8745,0.3581,1.2431,1.0200,0.9348,0.5723,0.4384


In [66]:
modelo.fit(Xt,yt)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
print(roc_auc_score(y_score=modelo.predict_proba(Xt)[:,1],y_true=yt))
print(roc_auc_score(y_score=modelo.predict_proba(Xv)[:,1],y_true=yv))

## Transformación Scoring

In [ ]:
betas = modelo.coef_[0].tolist()
alpha = modelo.intercept_[0]

In [ ]:
alpha

score_base(score al cual vamos a alinear los momios base), odds_base (momios_base),pdo (points to double the odd's)

In [ ]:
score_base = 137
odd_base = 2
pdo = 20

$factor = pdo/\log(2)\newline
offset = score\_base-factor\log(odds\_base)
\newline
pts = \left(-WoE\cdot\beta+\alpha/n\right)\cdot factor+offset/n
$

In [ ]:
factor = pdo/np.log(2)
offset = score_base-factor*np.log(odd_base)

In [ ]:
Xt[tgt] = yt
Xv[tgt] = yv

In [ ]:
X = pd.concat([Xt,Xv],ignore_index=True)

In [ ]:
n = len(varw)

In [ ]:
for b,v in zip(betas,varw):
    X[f'pts_{v}'] = np.ceil((-X[v]*b+alpha/n)*factor+offset/n).astype(int)

In [ ]:
X['score'] = X.filter(like='pts_').sum(axis=1)

In [ ]:
X.score.hist()

In [ ]:
X['r_score'] = pd.cut(X['score'],bins=range(0,360+36,36)).astype(str)

In [ ]:
#X.pivot_table(index='r_score',columns=tgt,values='score',aggfunc='count',fill_value=0).to_clipboard()

In [ ]:
varw

In [ ]:
l = []
for v,d in mapa_woe:
    if 'w_%s'%v in varw:
        d = {y:x for x,y in d.items()}
        aux = X[['w_%s'%v,'pts_w_%s'%v]].copy()
        aux.drop_duplicates(inplace=True)
        aux['w_%s'%v].replace(d,inplace=True)
        aux.rename(columns={'w_%s'%v:'atributo','pts_w_%s'%v:'puntos'},inplace=True)
        aux['característica'] = v
        l.append(aux)

In [ ]:
scorecard = pd.concat(l,ignore_index=True)

In [ ]:
with pd.ExcelWriter('scorecard.xlsx') as xl:
    scorecard.groupby(['característica','atributo']).max().to_excel(xl,sheet_name='scorecard')
    xl.close()